In [300]:
%matplotlib inline

import os, subprocess, tarfile
from glob import glob
import numpy as np
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cftime
import cartopy
import cartopy.crs as ccrs
import scipy.stats as stats
from scipy.io import loadmat

import string
alphabet=list(string.ascii_lowercase)

import warnings
warnings.filterwarnings('ignore')

seconds_per_day=60.*60.*24.
seconds_per_year=365.*seconds_per_day
carbon_molar_mass=12.011
cobalt_n_2_c = 106./16.

In [301]:
import matplotlib.colors as colors
class MidpointNormalize(colors.Normalize):
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        super().__init__(vmin, vmax, clip)

    def __call__(self, value, clip=None):
        # I'm ignoring masked values and all kinds of edge cases to make a
        # simple example...
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))

# Read in Data

In [315]:
case={}

#control
case['path_root']='/archive/Jessica.Luo/gz_test/MOM6_SIS2_COBALT'
case['machine_target']='gfdl.ncrc4-intel19-prod'
case['name']='OM4p5_CORE2_IAF_gzCOBALT-013022_cy5'

#cases
case['path_root']='/archive/Jessica.Luo/gz_test/MOM6_SIS2_gzCOBALTv2'
case['machine_target']='gfdl.ncrc4-intel19-prod'

case['name']='OM4p5_CORE2_IAF_gzCOBALT-tun_cy5'
case['name']='OM4p5_CORE2_IAF_gzCOBALT-tun_hp_cy5'
case['name']='OM4p5_CORE2_IAF_gzCOBALT-hp_cy5'


In [303]:
# define years for cycle
cycle=1
if 'cy1' in case['name']:
    cycle=1
if 'cy2' in case['name']:
    cycle=2
if 'cy3' in case['name']:
    cycle=3
if 'cy4' in case['name']:
    cycle=4
if 'cy5' in case['name']:
    cycle=5
    
print(cycle)

5


In [304]:
start_date=60*(cycle-1)+1
end_date=60*cycle+1
print(start_date,end_date)

241 301


In [305]:
short_casename=case['name'].replace('OM4p5_CORE2_IAF_','')
if short_casename=="gzCOBALT-060822":
    fig_casename='gzCOBALT-tun_cy1'
else:
    fig_casename=short_casename
print(short_casename, fig_casename)

gzCOBALT-013022_cy5 gzCOBALT-013022_cy5


In [306]:
# read in files
diagType = 'ocean_cobalt_omip_2d'
varlist = ['intpp','epc100']

prefix = os.path.join(case['path_root'], case['name'], case['machine_target'], 'pp', diagType, 'ts', 'annual', '5yr', diagType)
files_a = glob('.'.join([prefix, '*', 'intpp', 'nc']))
files_b = glob('.'.join([prefix, '*', 'epc100', 'nc']))
files_a.extend(files_b)
print(files_a)

['/archive/Jessica.Luo/gz_test/MOM6_SIS2_COBALT/OM4p5_CORE2_IAF_gzCOBALT-013022_cy5/gfdl.ncrc4-intel19-prod/pp/ocean_cobalt_omip_2d/ts/annual/5yr/ocean_cobalt_omip_2d.1948-1952.intpp.nc', '/archive/Jessica.Luo/gz_test/MOM6_SIS2_COBALT/OM4p5_CORE2_IAF_gzCOBALT-013022_cy5/gfdl.ncrc4-intel19-prod/pp/ocean_cobalt_omip_2d/ts/annual/5yr/ocean_cobalt_omip_2d.1953-1957.intpp.nc', '/archive/Jessica.Luo/gz_test/MOM6_SIS2_COBALT/OM4p5_CORE2_IAF_gzCOBALT-013022_cy5/gfdl.ncrc4-intel19-prod/pp/ocean_cobalt_omip_2d/ts/annual/5yr/ocean_cobalt_omip_2d.1958-1962.intpp.nc', '/archive/Jessica.Luo/gz_test/MOM6_SIS2_COBALT/OM4p5_CORE2_IAF_gzCOBALT-013022_cy5/gfdl.ncrc4-intel19-prod/pp/ocean_cobalt_omip_2d/ts/annual/5yr/ocean_cobalt_omip_2d.1963-1967.intpp.nc', '/archive/Jessica.Luo/gz_test/MOM6_SIS2_COBALT/OM4p5_CORE2_IAF_gzCOBALT-013022_cy5/gfdl.ncrc4-intel19-prod/pp/ocean_cobalt_omip_2d/ts/annual/5yr/ocean_cobalt_omip_2d.1968-1972.intpp.nc', '/archive/Jessica.Luo/gz_test/MOM6_SIS2_COBALT/OM4p5_CORE2_IAF_g

In [307]:
grid_file=os.path.join(case['path_root'], case['name'], case['machine_target'], 'pp')+'/ocean_annual/ocean_annual.static.nc'
ds=xr.open_mfdataset(files_a, combine='by_coords', decode_times=False)
grid=xr.open_dataset(grid_file)

# Compute Global NPP

In [308]:
ds.intpp

<xarray.DataArray 'intpp' (time: 60, yh: 576, xh: 720)>
dask.array<concatenate, shape=(60, 576, 720), dtype=float32, chunksize=(5, 576, 720), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) float64 182.5 547.5 912.5 ... 2.099e+04 2.135e+04 2.172e+04
  * xh       (xh) float64 -299.8 -299.2 -298.8 -298.2 ... 58.75 59.25 59.75
  * yh       (yh) float64 -77.91 -77.72 -77.54 -77.36 ... 89.47 89.68 89.89
Attributes:
    long_name:      Primary Organic Carbon Production by All Types of Phytopl...
    units:          mol m-2 s-1
    cell_measures:  area: areacello
    standard_name:  net_primary_mole_productivity_of_biomass_expressed_as_car...
    cell_methods:   area:mean yh:mean xh:mean time: mean
    time_avg_info:  average_T1,average_T2,average_DT

In [309]:
npp = (ds.intpp * grid.areacello).sum(dim=['yh', 'xh'])
npp = npp * seconds_per_year * carbon_molar_mass
print('net primary production: ' + str(np.format_float_scientific(npp.mean(dim='time').values,2)) + ' g C / yr')

net primary production: 5.39e+16 g C / yr


# Compute Global Carbon Export

In [310]:
ds.epc100

<xarray.DataArray 'epc100' (time: 60, yh: 576, xh: 720)>
dask.array<concatenate, shape=(60, 576, 720), dtype=float32, chunksize=(5, 576, 720), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) float64 182.5 547.5 912.5 ... 2.099e+04 2.135e+04 2.172e+04
  * xh       (xh) float64 -299.8 -299.2 -298.8 -298.2 ... 58.75 59.25 59.75
  * yh       (yh) float64 -77.91 -77.72 -77.54 -77.36 ... 89.47 89.68 89.89
Attributes:
    long_name:      Downward Flux of Particulate Organic Carbon
    units:          mol m-2 s-1
    cell_measures:  area: areacello
    standard_name:  sinking_mole_flux_of_particulate_organic_matter_expressed...
    cell_methods:   area:mean yh:mean xh:mean time: mean
    time_avg_info:  average_T1,average_T2,average_DT

In [311]:
export = (ds.epc100 * grid.areacello).sum(dim=['yh', 'xh'])
export = export * seconds_per_year * carbon_molar_mass
print('carbon export at 100 m: ' + str(np.format_float_scientific(export.mean(dim='time').values,2)) + ' g C / yr')

carbon export at 100 m: 6.34e+15 g C / yr


# Pull together into xarray dataset

In [312]:
global_npp = xr.DataArray(npp.values, coords={'time':np.arange(start_date,end_date)}, 
                             dims=['time'], name='npp', 
                             attrs={'name':'Total Global NPP','units':'g C y-1'})

global_export = xr.DataArray(export.values, coords={'time':np.arange(start_date,end_date)}, 
                             dims=['time'], name='expc100', 
                             attrs={'name':'Total Global Carbon Export at 100 m','units':'g C y-1'})

In [313]:
global_npp_export = global_npp.to_dataset()
global_npp_export['expc100'] = global_export


global_npp_export

<xarray.Dataset>
Dimensions:  (time: 60)
Coordinates:
  * time     (time) int64 241 242 243 244 245 246 ... 295 296 297 298 299 300
Data variables:
    npp      (time) float32 5.377e+16 5.414e+16 5.523e+16 ... 5.25e+16 5.232e+16
    expc100  (time) float32 6.325e+15 6.352e+15 ... 6.174e+15 6.165e+15

In [314]:
file_out='data/global_npp_export.'+fig_casename+'.nc'

if not os.path.exists(file_out):
    global_npp_export.to_netcdf(path=file_out, mode='w')
    print('writing to: '+file_out)
else:
    print(file_out + ' exists')

writing to: data/global_npp_export.gzCOBALT-013022_cy5.nc
